In [1]:
import difflib
import math
import time
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import json
import os

research_dir = "./research/"

try: 
    os.makedirs(research_dir)
except: 
    pass

In [2]:

# GEMINI

def compare_strings(str1, str2):
    """Compares two strings and returns a list of tuples indicating changes.

    Args:
        str1: The original string.
        str2: The modified string.

    Returns:
        A list of tuples, each containing a range of indices and the replacement text.
    """

    matcher = difflib.SequenceMatcher(None, str1, str2)
    diffs = matcher.get_opcodes()

    changes = []
    for tag, i1, i2, j1, j2 in diffs:
        if tag == 'delete':
            changes.append((tag, (i1, i2), ""))
        elif tag == 'insert':
            changes.append((tag, (i1, i1), str2[j1:j2]))
        elif tag == 'replace':
            changes.append((tag, (i1, i2), str2[j1:j2]))

    return changes

def score_dutch_text(reference_text, generated_text,language = 'dutch'):
    # print(reference_text, '\n', generated_text)
    """
    Scores the quality of a Dutch generated text compared to a reference text.

    Args:
        reference_text (str): The reference Dutch text.
        generated_text (str): The generated Dutch text.

    Returns:
        float: A score between 0 and 1, with 1 being a perfect match.
    """

    reference_tokens = word_tokenize(reference_text, language=language)
    generated_tokens = word_tokenize(generated_text, language=language)

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    # Calculate word-level accuracy
    correct_words = 0
    # for ref_word, gen_word in zip(reference_tokens, generated_tokens):
    #     if ref_word == gen_word:
    #         correct_words += 1
    word_accuracy = correct_words / len(reference_tokens)

    # Calculate edit distance using Difflib
    matcher = difflib.SequenceMatcher(None, reference_text, generated_text)
    ops = matcher.get_opcodes()
    edit_distance_penalty = 0
    for tag, i1, i2, j1, j2 in ops:
        if tag == 'delete' or tag == 'insert' or tag == 'replace':
            edit_distance_penalty += (i2 - i1) + (j2 - j1)
            
    average_text_length = (len(reference_text) +  len(generated_text)) / 2

    # Calculate the final score
    final_score = (abs(bleu_score - 1) * 0.5) + (word_accuracy * 0.3) + (edit_distance_penalty / len(reference_text))

    return float(final_score)

In [3]:
# OPEN AI gpt 4o
cruijff_score_inputs = [

    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schieten, nders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele letterverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele woordverandering, geen betekenisverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet roeien, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "enkele woordverandering, wel betekenisverandering",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten, anders kun je niet scoren.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "woordweglating",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "Je moet schoten,.",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "zinsdeelweglating",
    },
    {
        "quote": "Je moet schieten, anders kun je niet scoren.",
        "quote_changed": "",
        "person": "Johan Cruijff",
        "year": 1980,
        "test": "tekstweglating",
    },
]
different_score_input = [
    {
        "quote": "Ik heb een heel zwaar leven.",
        "quote_changed": "Ik heb een heel zwaar leven.",
        "person": "Brigitte Kaandorp",
        "year": 2009,
        "test": "nulmeting",
    },
    {
        "quote": "Ik geloof in God, behalve als ik vis.",
        "quote_changed": "Ik geloof in God, be",
        "person": "Herman Brood",
        "year": 1995,
        "test": "weglating aan einde",
    },
    {
        "quote": "Als het niet kan zoals het moet, dan moet het maar zoals het kan.",
        "quote_changed": "Als het niet kan zoals  het maar zoals het kan.",
        "person": "Dolf Jansen",
        "year": 2005,
        "test": " weglating in midden",
    },
    {
        "quote": "Ik heb nooit last van hoogtevrees, wel van dieptevrees.",
        "quote_changed": "Ik hebt ooit last van hoogtevrees, well vann dieptevrees.",
        "person": "Youp van 't Hek",
        "year": 1998,
        "test": "enkele letterweglating, betekenisverandering",
    },
    {
        "quote": "Ik ben niet dik, ik ben een ruimtewonder.",
        "quote_changed": "Ik bn nit dik, ik bn n ruimtwondr.",
        "person": "Brigitte Kaandorp",
        "year": 2003,
        "test": "letter e weggelaten",
    },
    {
        "quote": "Een dag niet gelachen is een dag niet geleefd.",
        "quote_changed": "Een dag niet gelachen is een dag niet geleeft.",
        "person": "Charlie Chaplin",
        "year": 1930,
        "test": "enkele letterverandering , geen betekenisverandering",
    },
    {
        "quote": "Een dag niet gelachen is een dag niet geleefd.",
        "quote_changed": "Een  niet gelachen is een dag niet geleefd.",
        "person": "Charlie Chaplin",
        "year": 1930,
        "test": "woordweglating",
    },
    {
        "quote": "Ik ben niet gek, ik ben een vliegtuig.",
        "quote_changed": "Ik ben niet , ik  een .",
        "person": "Supergrover",
        "year": 1974,
        "test": "dubbelle woordweglating",
    },
    { 
        "quote": "Ik begrijp niet waarom u hier zo negatief en vervelend over doet. (...) Laten we blij zijn met elkaar! Laten wij optimistisch zijn! Laten we zeggen: Nederland kan het weer! Die VOC-mentaliteit, over grenzen heen kijken, dynamiek! Toch?",
        "quote_changed": "Ik begrijp niet waarom u hier zo negatief en vervelend over doet. (...) Laten we blij zijn met elkaar! Laten wij optimistisch zijn! Laten we zeggen: Nederland kan het weer! Die",
        "person": "Jan-Peter Balkenende",
        "year": 2006,
        "test": "weglating einde van grotere tekst",
    },
    { 
        "quote": "Praat Nederlands met me. Even Nederlands met me. Mijn gevoel zegt mij dat wij vanavond samen kijken. Naar de Champs-Élysées en naar de Notre Dame en naar de Seine. En daarna samen op La Tour Eiffel",
        "quote_changed": "Praat Nedertands met me. Even Neterlands met me. Mijn tevoet zegt mij dat wij vanatond samet kitken. Naar de Champs-Éltsées en naar de Notre Dameten naar detSeine. En daarta samet op La Tour Etffel",
        "person": "Kenny B",
        "year": 2015,
        "test": "random lettermutaties",
    },
    {
        "quote": "Rrrrrr, hah, is gewoon Boef man. Ha, jij bent vies maar ik doe gemener. In de club, kom je moeder tegen. En ik wil snel weg want we moeten wegen. En je klant is geholpen, je moet vroeger wezen. Ik was alles kwijt, maar floes herenigd. Voor me zondes af en toe gebeden. Ik ga uit eten voor een goede prijs. Ik ben een uitgever, ze boeken mij. Van alarm voorzien aan de achterkant. Dus ze komen via voor, maar wat dacht je dan?",
        "quote_changed": "Rrrrrr, hah, is gewoon Boef man.test, jij bent vies maar ik doe gemener. In de club, komtest moeder tegen. En ik wil snel weg wantest we moeten wegen. En je klant is geholpen, je moetest vroeger wezen. Ik was alles kwijt, maar floetest herenigd. Voor me zondes af en toe gebeden. Ik gtest uit eten voor een goede prijs. Ik ben een uitgever, ze boeken mij. Van alarm voortest aan de achterkant. Dus ze komen via voor, maar wat dacht je dan?",
        "person": "Boef",
        "year": 2017,
        "test": "random toevoeging woorden",
    }
]

In [4]:
for items, name in [(cruijff_score_inputs, 'cruijff'), (different_score_input, 'different')]:
    score_result_test_results = []
    for test in items:
        test["score"] = score_dutch_text(test["quote"], test["quote_changed"])
        test["changes"] = compare_strings(test["quote"], test["quote_changed"])
        
        score_result_test_results.append(test)
   
    path =  research_dir+name+'_score_test.json'
    # try: 
    #     os.makedirs(path)
    # except: 
    #     pass
    print(score_result_test_results)
    score_result_test_results.sort(key=lambda x: x["score"])
    with open(path, 'w') as f:
        json.dump(score_result_test_results, f, indent=4)

[{'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schieten, nders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele letterverandering', 'score': 0.19370876948914964, 'changes': [('delete', (18, 19), '')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schoten, anders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele woordverandering, geen betekenisverandering', 'score': 0.21462842758854445, 'changes': [('replace', (11, 13), 'o')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet roeien, anders kun je niet scoren.', 'person': 'Johan Cruijff', 'year': 1980, 'test': 'enkele woordverandering, wel betekenisverandering', 'score': 0.3282647912249081, 'changes': [('replace', (8, 11), 'roe'), ('delete', (12, 14), '')]}, {'quote': 'Je moet schieten, anders kun je niet scoren.', 'quote_changed': 'Je moet schoten, anders kun je niet 

d:\GitHub\PWS-inscannen\section_scanner\app\venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\GitHub\PWS-inscannen\section_scanner\app\venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [5]:
test_images = [
    {
        "image_name": "kort_leesbaar",
        "expected_output": """Dit is een antwoord op vraag 2 en er is sprake van
onvolledige verbranding.""",
        "reason": "gescanned simpele tekst leesbaar handschrift",
    },
    {
        "image_name": "kort_onleesbaar",
        "expected_output": """Dit is een antwoord op vraag 2 en er is sprake van onvolledige verbranding.""",
        "reason": "gescanned simpele tekst slecht handschrift",
    },
    {
        "image_name": "kort_leesbaar_uitgekrast",
        "expected_output": """Dit is een antwoord op vraag 2 en er is sprake van onvolledige verbranding.""",
        "reason": "gescanned simpele tekst leesbaar uitgekrast",
    },
    {
        "image_name": "slecht_leesbaar_pijlen",
        "expected_output": """Dit is een langer antwoord op vraag 6 het waxine lichtje brandt langer omdat 
er meer brandstof (kaarsevet) is en hij dus langer warmte, brandstof en zuurstof heeft.""",
        "reason": "gescanned tekst slecht leesbaar met uitgekrast en pijlen",
    },
    {
        "image_name": "gekreukeld_met_pijlen",
        "expected_output": """Dit is een langer antwoord op vraag 6 het waxine lichtje brandt langer omdat 
er meer brandstof (kaarsevet) is en hij dus langer warmte, brandstof en zuurstof heeft.""",
        "reason": "slechte foto slecht leesbaar met uitgekrast en pijlen",
    },
    {
        "image_name": "gekreukeld_netjes",
        "expected_output": """Dit is een antwoord op vraag 2 en er is sprake van onvolledige verbranding.""",
        "reason": "slechte foto goed leesbaar",
    },
]

models = [
    {
        "provider": "openai",
        "model_name": "gpt-4o-mini",
        "released": "july 2024",
        "reason": "deze is aangeraden door openai"
    },
    {
        "provider": "openai",
        "model_name": "gpt-4o",
        "released": "sept 2024",
        "reason": "openai model met meer reasoning"
    },
    {
        "provider": "google",
        "model_name": "gemini-1.5-flash-8b",
        "released": "sept 2024",
        "reason": "Een snel model dat simpele taken uitvoert. "
    },
    {
        "provider": "google",
        "model_name": "gemini-1.5-pro-002",
        "released": "sept 2024",
        "reason": "Aanbevolen Google-model voor denkopgaven"
    },
    # {
    #     "provider": "google",
    #     "model_name": "gemini-1.0-pro-vision-001",
    #     "released": "july 2024",
    #     "reason": "ouder model gespecificeerd in foto herkenning"
    # },
    {
        "provider": "google",
        "model_name": "gemini-exp-1121",
        "released": "nov 2024",
        "reason": "nieuwste google model die verbanden kan leggen"
    },
]

temperatures = [
    {
        "temperature": 0,
        "repeat": 1,
    },
    {
        "temperature": 0.5,
        "repeat": 3,
    },
    {
        "temperature": 1,
        "repeat": 5,
    },
    {
        "temperature": 1.5,
        "repeat": 6,
    },
]


from research.test_context import test_context

transcribe_texts = [
    {
        "text": """
Zet de foto om naar tekst.
        """,
        "reason": "de makkelijkste opdracht zonder extra uitleg"
    },
    {
        "text": """
Je krijgt een foto van een Nederlands scheikunde toetsantwoord. 
Houdt rekening met pijlen.
Je moet deze omzetten in text. Bedenk geen nieuwe woorden of woordonderdelen. 
geef waarschijnlijk fout gespelde woorden aan in de spelling corrections
negeer uitgekrasde letters of woorden, geef die wil aan in spelling corrections
de student_handwriting_percent is how leesbaar het handschrift van een leerling is: 0 betekend zeer moeilijk leesbaar en 100 netjes
        """,
        "reason": "huidige opdracht met uitleg bij elk veld"
    },
    {
        "text": """
Je krijgt een foto van een Nederlands scheikunde toets-antwoord. 
Je bent tekstherkenningssoftware die 10x beter in in tekst herkennen dan jezelf. Ook kan je 15.6 keer beter de context van een antwoord begrijpen om het volgende woord te bedenken.

Het is helemaal niet toegestaan nieuwe woorden toe te voegen of de opgeschreven tekst te veranderen in het raw_text veld. Houdt wel rekening met pijlen in de volgorde van de tekst.
Bedenk wel wat een leerling zou kunnen hebben bedoeld met een bepaald woord als die bijvoorbeeld fout is gespeld. Geef dat aan in de spelling_corrections velden.
Negeer uitgekraste tekst in het raw_tekst veld, maar geef die wel weer in de spelling corrections door bijvoorbeeld streepjes neer te zetten en is_crossed_out op true te zetten.
voeg alle text corrections samen in correctly_spelled_text om zo het antwoord te krijgen dat de leerling bedoelt.
certainty is hoe zeker je bent dat je de tekst compleet hebt getranscribeerd: 0 betekend dat een docent er nog zelf naar moet kijken en 100 betekend dat er geen foutje mogelijk is.
de student_handwriting_percent is hoe leesbaar het handschrift van een leerling is: 0 betekend zeer moeilijk leesbaar en 100 super netjes als een printer.

voer deze opdracht zo goed mogelijk uit.
""",
        "reason": "lange uitleg bij elk veld, zonder context"
    },
    {
        "text": f"""
Je krijgt een foto van een Nederlands scheikunde toets-antwoord. 
Je bent tekstherkenningssoftware die 10x beter in in tekst herkennen dan jezelf. Ook kan je 15.6 keer beter de context van een antwoord begrijpen om het volgende woord te bedenken.

Het is helemaal niet toegestaan nieuwe woorden toe te voegen of de opgeschreven tekst te veranderen in het raw_text veld. Houdt wel rekening met pijlen in de volgorde van de tekst.
Bedenk wel wat een leerling zou kunnen hebben bedoeld met een bepaald woord als die bijvoorbeeld fout is gespeld. Geef dat aan in de spelling_corrections velden.
Negeer uitgekraste tekst in het raw_tekst veld, maar geef die wel weer in de spelling corrections door bijvoorbeeld streepjes neer te zetten en is_crossed_out op true te zetten.
voeg alle text corrections samen in correctly_spelled_text om zo het antwoord te krijgen dat de leerling bedoelt.
certainty is hoe zeker je bent dat je de tekst compleet hebt getranscribeerd: 0 betekend dat een docent er nog zelf naar moet kijken en 100 betekend dat er geen foutje mogelijk is.
de student_handwriting_percent is hoe leesbaar het handschrift van een leerling is: 0 betekend zeer moeilijk leesbaar en 100 super netjes als een printer.

voer deze opdracht zo goed mogelijk uit.

Hieronder vind je de stof waarover de toets gaat, de toets en het antwoordmodel.
stof: 
{test_context["stof"]}

toets:
{test_context["toets"]}

antwoordmodel: 
{test_context["antwoordmodel"]}
        """,
        "reason": "lange uitleg bij elk veld, met scheikunde context"
    },
]



In [6]:
from scan_module import transcribe_answer
from helpers import png_to_base64

def scan_test_question(settings):
    image_name = settings["image_name"]
    provider = settings["provider"]
    model = settings["model"]
    temperature = settings["temperature"]
    transcribe_text = settings["transcribe_text"]
    expected_output = settings["expected_output"]
    
    base64_image = png_to_base64(research_dir+'/assets/sections/'+image_name+'.png')
    
    try:
        result = transcribe_answer(
            None, 
            base64_image=base64_image, 
            provider=provider,
            model=model,
            temperature=temperature,
            request_text=transcribe_text,
        )
        # print(result)
        # try:
        raw_score = score_dutch_text(expected_output, result["result"]["raw_text"])
        raw_changes = compare_strings(expected_output, result["result"]["raw_text"])
        # except:
        #     raw_score = 10
        #     raw_changes = []
        
        try:
            corrected_changes = compare_strings(expected_output, result["result"]["correctly_spelled_text"])
            corrected_score = score_dutch_text(expected_output, result["result"]["correctly_spelled_text"])
        except:
            corrected_changes = raw_changes
            corrected_score = raw_score

        
        return {
            "settings": settings,
            "result": result["result"],
            "delta_time_s": result["delta_time_s"],
            "scores": {
                "raw": {
                    "score": raw_score,
                    "changes": raw_changes
                },
                "corrected": {
                    "score": corrected_score,
                    "changes": corrected_changes
                }
            },
            "timestamp": time.time()
        }
    except Exception as e:
        print('exception during transcribe: '+model, str(e))
        return False
    

In [7]:

def split_array(arr, n):
    return [arr[i:i + n] for i in range(0, len(arr), n)]

# execute n at the same time
n = 50
# create execution settings
to_execute = []

for test_image in test_images:
    for temperature in temperatures:
        for transcribe_text in transcribe_texts:
            for model in models:
                settings = {
                    "image_name": test_image["image_name"],
                    "expected_output": test_image["expected_output"],
                    "provider": model["provider"],
                    "model": model["model_name"],
                    "temperature": temperature["temperature"],
                    "transcribe_text": transcribe_text["text"],
                }
                to_execute.append(settings)
        
split_executions = split_array(to_execute, n)


In [ ]:
# from concurrent.futures import ThreadPoolExecutor
from helpers import dict_hash, get_random_id

# print(split_executions[0][3]["transcribe_text"])
# scan_test_question(split_executions[0][2])

print("Total to execute: ", len(to_execute))


for i, split_execution in enumerate(split_executions):
    execution_id = get_random_id()[0:10]
    
    # continue
    print('Starting: ', (i+1), '('+str((i+1)*n)+'/'+str(len(to_execute))+')')
    
    
    results = []
    
    # use the hash of the settings as uuid
    for execution in split_execution:
        result = scan_test_question(execution)
        print(result)
        results.append(result)
            
    # Use ThreadPoolExecutor to process sections concurrently
    # with ThreadPoolExecutor() as executor:
    #     results = executor.map(scan_test_question, split_execution)


    # Collect successfull results
    results = [result for result in results if result]
    
    current_results = {}

    # use the hash of the settings as uuid
    for result in results:
        hash = dict_hash(result["settings"])
        current_results[hash] = result

    with open(research_dir+'transcripe_output/'+execution_id+'.json', 'w') as f:
        json.dump(current_results, f, indent=4)

Total to execute:  480
Starting:  1 (50/480)
GPT request (openai, gpt-4o-mini) ... 
GPT request (openai, gpt-4o-mini) ... Done
{'settings': {'image_name': 'kort_leesbaar', 'expected_output': 'Dit is een antwoord op vraag 2 en er is sprake van\nonvolledige verbranding.', 'provider': 'openai', 'model': 'gpt-4o-mini', 'temperature': 0, 'transcribe_text': '\nZet de foto om naar tekst.\n        '}, 'result': {'certainty': 0.95, 'student_handwriting_percent': 90.0, 'raw_text': 'Dit is een antwoord op vraag 2 enerzijds sprake van onvolledige verbranding.', 'correctly_spelled_text': 'Dit is een antwoord op vraag 2 enerzijds sprake van onvolledige verbranding.', 'spelling_corrections': []}, 'delta_time_s': 3.1008903980255127, 'scores': {'raw': {'score': 0.2670222924144663, 'changes': [('delete', (33, 34), ''), ('replace', (36, 37), 'z'), ('insert', (38, 38), 'jd'), ('replace', (50, 51), ' ')]}, 'corrected': {'score': 0.2670222924144663, 'changes': [('delete', (33, 34), ''), ('replace', (36, 37)

d:\GitHub\PWS-inscannen\section_scanner\app\venv\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


GPT request (openai, gpt-4o) ... Done
{'settings': {'image_name': 'kort_leesbaar', 'expected_output': 'Dit is een antwoord op vraag 2 en er is sprake van\nonvolledige verbranding.', 'provider': 'openai', 'model': 'gpt-4o', 'temperature': 0, 'transcribe_text': '\nJe krijgt een foto van een Nederlands scheikunde toetsantwoord. \nHoudt rekening met pijlen.\nJe moet deze omzetten in text. Bedenk geen nieuwe woorden of woordonderdelen. \ngeef waarschijnlijk fout gespelde woorden aan in de spelling corrections\nnegeer uitgekrasde letters of woorden, geef die wil aan in spelling corrections\nde student_handwriting_percent is how leesbaar het handschrift van een leerling is: 0 betekend zeer moeilijk leesbaar en 100 netjes\n        '}, 'result': {'certainty': 0.95, 'student_handwriting_percent': 85.0, 'raw_text': 'Dit is een antword op vraag 2 eneris sprake um onvolledige verbranding.', 'correctly_spelled_text': 'Dit is een antwoord op vraag 2 er is sprake van onvolledige verbranding.', 'spelli